# By: Luc Rieffel
# CPSC 390 Project 3

In [69]:
import pandas as pd

# Import Dataset
sms_message_spam = pd.read_table('SMS', sep='\t', header=None,names=['Label', 'SMS'])

print(sms_message_spam.shape)
sms_message_spam.head()

(5572, 2)


,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [70]:
# Randomize the dataset
data_randomized = sms_message_spam.sample(frac=1, random_state=1)

# Calculate index for split
training_test_index = round(len(data_randomized) * 0.8)

# Split into training and test sets
training_set = data_randomized[:training_test_index].reset_index(drop=True)
test_set = data_randomized[training_test_index:].reset_index(drop=True)

X_train = training_set['Label']
y_train = training_set['Label']
X_test = test_set['SMS']
y_train = test_set['SMS']

print(training_set.shape)
print(test_set.shape)

(4458, 2)
(1114, 2)


In [71]:
training_set['SMS'] = training_set['SMS'].str.replace(
   '\W', ' ') # Removes punctuation
training_set['SMS'] = training_set['SMS'].str.lower()
training_set

/var/folders/d9/pw3fpxsn6z36z5c4m9bl4g140000gn/T/ipykernel_46223/286274972.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  training_set['SMS'] = training_set['SMS'].str.replace(


,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...
...,...,...
4453,ham,sorry i ll call later in meeting any thing re...
4454,ham,babe i fucking love you too you know fuck...
4455,spam,u ve been selected to stay in 1 of 250 top bri...
4456,ham,hello my boytoy geeee i miss you already a...


In [72]:
training_set['SMS'] = training_set['SMS'].str.split()

vocabulary = []
for sms in training_set['SMS']:
    for word in sms:
        vocabulary.append(word)
vocabulary = list(set(vocabulary)) ### only count the number of unique words
print(len(vocabulary))

7783


In [73]:
word_counts_per_sms = {unique_word: [0] * len(training_set['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(training_set['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [74]:
# from sklearn.feature_extraction.text import CountVectorizer



# vectorizer = CountVectorizer()
# X_train = vectorizer.fit_transform(training_set["SMS"])
# X_test = vectorizer.transform(testing_set["SMS"])

In [75]:
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,gf,anytime,boston,singles,nauseous,connect,pest,roommates,emergency,m8,...,rimac,09061104283,all,studying,ltd,converter,jolt,tablet,related,lyrics
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0


In [76]:
training_new = pd.concat([training_set, word_counts], axis=1)
training_new.head()

,Label,SMS,gf,anytime,boston,singles,nauseous,connect,pest,roommates,...,rimac,09061104283,all,studying,ltd,converter,jolt,tablet,related,lyrics
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0


# You will start from here.

In [77]:
#theoretical steps
#calculate the probability of spam and ham given the text
#calculate the probability of spam and ham given the text
#compare the two probabilities and assign the text to the class with the highest probability
#use bayes theorem by switching A and B
#count the frequency of each word in spam and ham

In [78]:
# Hints:
# Laplace smoothing
alpha = 1
# Step 1: you need to caculate P(Spam) and P(Ham)
# Isolating spam and ham messages first
spam_messages = training_new[training_new['Label'] == 'spam']
ham_messages = training_new[training_new['Label'] == 'ham']
# P(Spam) and P(Ham)
p_spam = len(spam_messages) / len(training_new)
p_ham = len(ham_messages) / len(training_new)

# Step 2: you need to count N_Spam, N_Ham
# N_Spam
n_words_per_spam_message = spam_messages['SMS'].apply(len)
n_spam = n_words_per_spam_message.sum()
# N_Ham
n_words_per_ham_message = ham_messages['SMS'].apply(len)
n_ham = n_words_per_ham_message.sum()
# N_Vocabulary
n_vocabulary = len(vocabulary)



In [79]:
# Step 3: you need to count the number of times the word w occurs in spam/ham message: N_w_spam
#get all unique words in vocabulary and add to a dictionary
parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_ham = {unique_word:0 for unique_word in vocabulary}

# Calculate parameters of bayes equation
for word in vocabulary:
    n_word_given_spam = spam_messages[word].sum() # spam_messages already defined
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocabulary)
    parameters_spam[word] = p_word_given_spam

    n_word_given_ham = ham_messages[word].sum() # ham_messages already defined
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
    parameters_ham[word] = p_word_given_ham

In [80]:
import re
# Step 4: then you can calculate the prob of occurance of each word: p(w|spam)=(N_w_spam+alpha)/(N_Spam+alpha*N_Vocabulary)p(w|Ham)=(N_w_ham+alpha)/(N_Ham+alpha*N_Vocabulary)
def classify(message):
    '''
    message: some string
    '''
    message = re.sub('\W', ' ', message) #use regex to remove whitespace from string
    message = message.lower().split() #make lowercase and split
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
        if word in parameters_ham: 
            p_ham_given_message *= parameters_ham[word]
    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else: 
        print("Label: Spam")

In [81]:
classify('You are a winner!! This is the secret code to receive your money: CC92043.')

P(Spam|message): 9.652535728893754e-35
P(Ham|message): 9.327759475533778e-38
Label: Spam


In [82]:
# Step 5: Now do the prediction if it is spam or ham for each sentence, then store your prediction results (1 or 0 ) to test['predicted']
def classify_test_set(message):
    '''
    message: some string
    '''
    message = re.sub('\W', ' ', message)
    message = message.lower().split()
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]

        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else: 
        return 'spam'

In [83]:
test_set['predicted'] = test_set['SMS'].apply(classify_test_set)
test_set.head(20)

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham
5,ham,But my family not responding for anything. Now...,ham
6,ham,U too...,ham
7,ham,Boo what time u get out? U were supposed to ta...,ham
8,ham,Genius what's up. How your brother. Pls send h...,ham
9,ham,I liked the new mobile,ham


# Calculate accuracy, precision, recall and F1_score. 

In [84]:
# Model Evaluation with our calculations
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('Accuracy score: {}'.format(accuracy_score(test_set['Label'], test_set['predicted'])))
print('Precision score: {}'.format(precision_score(test_set['Label'], test_set['predicted'], average = None)[0]))
print('Recall score: {}'.format(recall_score(test_set['Label'], test_set['predicted'], average = None)[0]))#,average='micro')
print('F1 score: {}'.format(f1_score(test_set['Label'], test_set['predicted'], average = None)[0]))

Accuracy score: 0.9874326750448833
Precision score: 0.9917440660474717
Recall score: 0.9937952430196484
F1 score: 0.9927685950413223


### Now  use function MultinomialNB (from sklearn.naive_bayes import MultinomialNB) to validate your results, check the accuracy

In [85]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

X_train = training_new[training_new.columns[2:]]
y_train = training_new["Label"]

In [86]:
print(X_train, y_train)

      gf  anytime  boston  singles  nauseous  connect  pest  roommates  \
0      0        0       0        0         0        0     0          0   
1      0        0       0        0         0        0     0          0   
2      0        0       0        0         0        0     0          0   
3      0        0       0        0         0        0     0          0   
4      0        0       0        0         0        0     0          0   
...   ..      ...     ...      ...       ...      ...   ...        ...   
4453   0        0       0        0         0        0     0          0   
4454   0        0       0        0         0        0     0          0   
4455   0        0       0        0         0        0     0          0   
4456   0        0       0        0         0        0     0          0   
4457   0        0       0        0         0        0     0          0   

      emergency  m8  ...  rimac  09061104283  all  studying  ltd  converter  \
0             0   0  ...      0 

In [89]:
model = MultinomialNB()
model.fit(X_train, y_train)
word_counts_per_sms = {unique_word: [0] * len(test_set["SMS"]) for unique_word in vocabulary} #get word counts
for index, sms in enumerate(test_set["SMS"]):
    for word in sms: 
        if word in vocabulary: 
            word_counts_per_sms[word][index] += 1 #add to appropriate index in test set
test_word_counts = pd.DataFrame(word_counts_per_sms)
test_word_counts

,gf,anytime,boston,singles,nauseous,connect,pest,roommates,emergency,m8,...,rimac,09061104283,all,studying,ltd,converter,jolt,tablet,related,lyrics
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1109,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1110,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1111,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1112,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [90]:
test_set["sklearn_preds"] = model.predict(test_word_counts)#get predictions
print('Accuracy score: {}'.format(accuracy_score(test_set['Label'], test_set['sklearn_preds'])))
print('Precision score: {}'.format(precision_score(test_set['Label'], test_set['sklearn_preds'], average = None)[0]))
print('Recall score: {}'.format(recall_score(test_set['Label'], test_set['sklearn_preds'], average = None)[0]))#,average='micro')
print('F1 score: {}'.format(f1_score(test_set['Label'], test_set['sklearn_preds'], average = None)[0]))

Accuracy score: 0.8671454219030521
Precision score: 0.8685868586858686
Recall score: 0.9979317476732161
F1 score: 0.9287776708373435
